In [1]:
import numpy as np
import pandas as pd
from keras.applications.vgg19 import VGG19
from keras.applications.resnet import ResNet50
from keras.applications.inception_v3 import InceptionV3
from glob import glob
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, GlobalAveragePooling2D, MaxPooling2D
from keras.models import Sequential, Model, load_model
from os.path import join
from json import dump, load

Using TensorFlow backend.


In [2]:
X_files = ['raw_X_Lasic.npy'] * 2 + ['raw_X_Lasic_180x180.npy', 'raw_X_Lasic_120x120.npy']
y_files = ['raw_y_Lasic.npy'] * 2 + ['raw_y_Lasic_180x180.npy', 'raw_y_Lasic_120x120.npy']
X_files

['raw_X_Lasic.npy',
 'raw_X_Lasic.npy',
 'raw_X_Lasic_180x180.npy',
 'raw_X_Lasic_120x120.npy']

In [3]:
def gen_lenet(img_height, img_width, num_classes):
    model = Sequential()
    model.add(Conv2D(32, (5, 5), activation='relu', input_shape=(img_height, img_width, 1))) 
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu')) 
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(num_classes, activation='softmax'))
    return model

def gen_vgg(img_height, img_width, num_classes):
    base_model = VGG19(weights=None, include_top=False, input_shape= (img_height,img_width,1))

    layer_dict = dict([(layer.name, layer) for layer in base_model.layers])
    x = layer_dict['block2_pool'].output
    x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(256, activation= 'relu')(x)
    x = Dense(256, activation= 'relu')(x)
    x = Dense(256, activation= 'relu')(x)
    predictions = Dense(num_classes, activation= 'softmax')(x)
    
    return Model(inputs = base_model.input, outputs = predictions)

def gen_inception(img_height, img_width, num_classes):
    base_model = InceptionV3(weights=None, include_top=False, input_shape=(img_height, img_width, 1))

    layer_dict = dict([(layer.name, layer) for layer in base_model.layers])
    x = layer_dict['mixed10'].output
    x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dense(256, activation='relu')(x)
    x = Dense(256, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    return Model(inputs=base_model.input, outputs=predictions)

def gen_resnet(img_height, img_width, num_classes):
    base_model = ResNet50(weights=None, include_top=False, input_shape= (img_height,img_width,1))

    layer_dict = dict([(layer.name, layer) for layer in base_model.layers])
    x = base_model.layers[-1].output
    x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dense(256, activation='relu')(x)
    x = Dense(256, activation='relu')(x)
    predictions = Dense(num_classes, activation= 'softmax')(x)
    
    return Model(inputs = base_model.input, outputs = predictions)

models = [gen_lenet, gen_vgg, gen_inception, gen_resnet]
models_names = ['LeNet', 'VGG19', 'InceptionV3', 'ResNet50']

In [ ]:
train_files = sorted(glob('Lasic_K_Fold/train_Lasic*'))
test_files = sorted(glob('Lasic_K_Fold/test_Lasic*'))
with open("results_lasic") as fp:
            results = load(fp)
for name, model_gen, X_file, y_file in zip(models_names, models, X_files, y_files):
    X = np.load(X_file)
    y = np.load(y_file)
    img_size = X.shape[1], X.shape[2]
    num_classes = len(np.unique(y))
    results.setdefault(name, {})
    for i, (train_file, test_file) in enumerate(zip(train_files, test_files), 1):
        i = str(i)
        if i in results.get(name, {}):
            print(f'In file {name} fold {i}:', results[name][i])
            continue
        train = np.load(train_file)
        test = np.load(test_file)
        model = model_gen(*img_size, num_classes)
        model.compile(optimizer='adam', # Optimization routine, which tells the computer how to adjust the parameter values to minimize the loss function.
              loss='sparse_categorical_crossentropy', # Loss function, which tells us how bad our predictions are.
              metrics=['accuracy']) # List of metrics to be evaluated by the model during training and testing.
        model.fit(X[train], y[train], epochs=100, batch_size=64, verbose=0)
        test_loss, test_acc = model.evaluate(X[test], y[test], verbose=0)
        print(f'{name} fold {i}:', test_acc)
        results[name][i] = test_acc
        with open("results_lasic", 'w') as fp:
            dump(results, fp)

In file LeNet fold 1: 0.9895833134651184
In file LeNet fold 2: 0.9958333373069763
In file LeNet fold 3: 0.9979166388511658
In file LeNet fold 4: 0.9916666746139526
In file LeNet fold 5: 1.0
In file LeNet fold 6: 0.9979166388511658
In file LeNet fold 7: 0.9937499761581421
In file LeNet fold 8: 0.9958333373069763
In file LeNet fold 9: 0.9979166388511658
In file LeNet fold 10: 0.9979166388511658
In file VGG19 fold 1: 0.9895833134651184
In file VGG19 fold 2: 0.9979166388511658
In file VGG19 fold 3: 0.9937499761581421
In file VGG19 fold 4: 0.9937499761581421
In file VGG19 fold 5: 1.0
In file VGG19 fold 6: 1.0
In file VGG19 fold 7: 0.9937499761581421
In file VGG19 fold 8: 0.9958333373069763
In file VGG19 fold 9: 0.9979166388511658
In file VGG19 fold 10: 0.9958333373069763
In file InceptionV3 fold 1: 0.9979166388511658
In file InceptionV3 fold 2: 0.9520833492279053
InceptionV3 fold 3: 0.21458333730697632
